# Smoking Cessation Study - Cox regression

12/12/2017 EPQ  Add TIMI and actual severity  

In [1]:
library(survival)

In [2]:
coxreg = read.csv("../whosein8.csv") #read the Cox regression and Leikert scale data from whosein_final.ipynb

str(coxreg)

'data.frame':	434 obs. of  49 variables:
 $ X                 : int  0 1 2 3 4 5 6 7 8 9 ...
 $ isg_no            : num  10587 10636 10657 10664 10671 ...
 $ event             : num  1 1 1 0 1 1 0 1 1 1 ...
 $ hours_to_event    : num  25.62 25.94 2.98 2088 114.74 ...
 $ subject_id_x      : num  22 23 25 26 27 28 29 31 34 35 ...
 $ baseline_b1_001   : num  5 4 4 2 3 5 5 4 3 3 ...
 $ baseline_c1_001   : num  4 4 4 4 3 5 5 4 3 3 ...
 $ baseline_d1_001   : num  4 4 3 4 1 4 4 5 2 5 ...
 $ baseline_b1_002   : num  1 4 3 2 3 5 5 2 3 3 ...
 $ baseline_c1_002   : num  3 3 3 4 3 5 5 4 3 4 ...
 $ baseline_d1_002   : num  2 3 3 4 2 4 3 4 2 5 ...
 $ baseline_b1_003   : num  3 3 4 4 3 1 1 2 4 1 ...
 $ baseline_c1_003   : num  3 3 3 2 2 1 1 1 3 1 ...
 $ baseline_d1_003   : num  1 3 3 2 3 4 3 2 3 1 ...
 $ baseline_b1_004   : num  4 3 3 2 3 5 5 3 2 5 ...
 $ baseline_c1_004   : num  2 3 3 4 3 5 4 1 3 5 ...
 $ baseline_d1_004   : num  4 3 3 4 2 5 3 4 2 5 ...
 $ subject_id_y      : num  22 23 25 26 27 28 

In [3]:
summary(coxreg)

       X             isg_no          event       hours_to_event    
 Min.   :  0.0   Min.   :10573   Min.   :0.000   Min.   :   1.241  
 1st Qu.:108.2   1st Qu.:11415   1st Qu.:1.000   1st Qu.:   7.825  
 Median :216.5   Median :12250   Median :1.000   Median :  43.542  
 Mean   :216.5   Mean   :12239   Mean   :0.797   Mean   : 319.065  
 3rd Qu.:324.8   3rd Qu.:13070   3rd Qu.:1.000   3rd Qu.: 203.172  
 Max.   :433.0   Max.   :13884   Max.   :1.000   Max.   :2088.000  
                                 NA's   :40      NA's   :40        
  subject_id_x   baseline_b1_001 baseline_c1_001 baseline_d1_001
 Min.   : 21.0   Min.   :1.000   Min.   :1.000   Min.   :1.000  
 1st Qu.:123.2   1st Qu.:3.000   1st Qu.:4.000   1st Qu.:2.000  
 Median :228.5   Median :4.000   Median :4.000   Median :4.000  
 Mean   :353.0   Mean   :3.788   Mean   :4.169   Mean   :3.384  
 3rd Qu.:614.2   3rd Qu.:5.000   3rd Qu.:5.000   3rd Qu.:4.500  
 Max.   :722.0   Max.   :5.000   Max.   :5.000   Max.   :5.000  
 

## Cox regression with principal component scores - perceived illness severity

## Compute principal components for baseline predictors

In [4]:
keeplist = c('isg_no','event','hours_to_event',
             'baseline_b1_001','baseline_b1_002','baseline_b1_003','baseline_b1_004',
             'baseline_c1_001','baseline_c1_002','baseline_c1_003','baseline_c1_004',
             'baseline_d1_001','baseline_d1_002','baseline_d1_003','baseline_d1_004')

perceived_severity = na.omit(coxreg[,keeplist])
str(perceived_severity)

'data.frame':	390 obs. of  15 variables:
 $ isg_no         : num  10587 10636 10657 10664 10671 ...
 $ event          : num  1 1 1 0 1 1 0 1 1 1 ...
 $ hours_to_event : num  25.62 25.94 2.98 2088 114.74 ...
 $ baseline_b1_001: num  5 4 4 2 3 5 5 4 3 3 ...
 $ baseline_b1_002: num  1 4 3 2 3 5 5 2 3 3 ...
 $ baseline_b1_003: num  3 3 4 4 3 1 1 2 4 1 ...
 $ baseline_b1_004: num  4 3 3 2 3 5 5 3 2 5 ...
 $ baseline_c1_001: num  4 4 4 4 3 5 5 4 3 3 ...
 $ baseline_c1_002: num  3 3 3 4 3 5 5 4 3 4 ...
 $ baseline_c1_003: num  3 3 3 2 2 1 1 1 3 1 ...
 $ baseline_c1_004: num  2 3 3 4 3 5 4 1 3 5 ...
 $ baseline_d1_001: num  4 4 3 4 1 4 4 5 2 5 ...
 $ baseline_d1_002: num  2 3 3 4 2 4 3 4 2 5 ...
 $ baseline_d1_003: num  1 3 3 2 3 4 3 2 3 1 ...
 $ baseline_d1_004: num  4 3 3 4 2 5 3 4 2 5 ...
 - attr(*, "na.action")=Class 'omit'  Named int [1:44] 12 84 113 276 395 396 397 398 399 400 ...
  .. ..- attr(*, "names")= chr [1:44] "12" "84" "113" "276" ...


In [5]:
perceived_severity2 = perceived_severity[,4:15]
str(perceived_severity2)

'data.frame':	390 obs. of  12 variables:
 $ baseline_b1_001: num  5 4 4 2 3 5 5 4 3 3 ...
 $ baseline_b1_002: num  1 4 3 2 3 5 5 2 3 3 ...
 $ baseline_b1_003: num  3 3 4 4 3 1 1 2 4 1 ...
 $ baseline_b1_004: num  4 3 3 2 3 5 5 3 2 5 ...
 $ baseline_c1_001: num  4 4 4 4 3 5 5 4 3 3 ...
 $ baseline_c1_002: num  3 3 3 4 3 5 5 4 3 4 ...
 $ baseline_c1_003: num  3 3 3 2 2 1 1 1 3 1 ...
 $ baseline_c1_004: num  2 3 3 4 3 5 4 1 3 5 ...
 $ baseline_d1_001: num  4 4 3 4 1 4 4 5 2 5 ...
 $ baseline_d1_002: num  2 3 3 4 2 4 3 4 2 5 ...
 $ baseline_d1_003: num  1 3 3 2 3 4 3 2 3 1 ...
 $ baseline_d1_004: num  4 3 3 4 2 5 3 4 2 5 ...


In [6]:
summary(perceived_severity2)

 baseline_b1_001 baseline_b1_002 baseline_b1_003 baseline_b1_004
 Min.   :1.000   Min.   :1.000   Min.   :1.000   Min.   :1.000  
 1st Qu.:3.000   1st Qu.:3.000   1st Qu.:1.000   1st Qu.:2.000  
 Median :4.000   Median :4.000   Median :2.000   Median :3.000  
 Mean   :3.785   Mean   :3.605   Mean   :2.505   Mean   :3.008  
 3rd Qu.:5.000   3rd Qu.:5.000   3rd Qu.:4.000   3rd Qu.:4.000  
 Max.   :5.000   Max.   :5.000   Max.   :5.000   Max.   :5.000  
 baseline_c1_001 baseline_c1_002 baseline_c1_003 baseline_c1_004
 Min.   :1.000   Min.   :1.000   Min.   :1.000   Min.   :1.000  
 1st Qu.:4.000   1st Qu.:3.000   1st Qu.:1.000   1st Qu.:2.000  
 Median :4.000   Median :4.000   Median :2.000   Median :4.000  
 Mean   :4.169   Mean   :3.959   Mean   :2.123   Mean   :3.492  
 3rd Qu.:5.000   3rd Qu.:5.000   3rd Qu.:3.000   3rd Qu.:5.000  
 Max.   :5.000   Max.   :5.000   Max.   :5.000   Max.   :5.000  
 baseline_d1_001 baseline_d1_002 baseline_d1_003 baseline_d1_004
 Min.   :1.000   Min.   :

In [7]:
perceived_severity2

,baseline_b1_001,baseline_b1_002,baseline_b1_003,baseline_b1_004,baseline_c1_001,baseline_c1_002,baseline_c1_003,baseline_c1_004,baseline_d1_001,baseline_d1_002,baseline_d1_003,baseline_d1_004
1,5,1,3,4,4,3,3,2,4,2,1,4
2,4,4,3,3,4,3,3,3,4,3,3,3
3,4,3,4,3,4,3,3,3,3,3,3,3
4,2,2,4,2,4,4,2,4,4,4,2,4
5,3,3,3,3,3,3,2,3,1,2,3,2
6,5,5,1,5,5,5,1,5,4,4,4,5
7,5,5,1,5,5,5,1,4,4,3,3,3
8,4,2,2,3,4,4,1,1,5,4,2,4
9,3,3,4,2,3,3,3,3,2,2,3,2
10,3,3,1,5,3,4,1,5,5,5,1,5


In [8]:
pc = princomp(perceived_severity2,scores=TRUE)

In [9]:
str(pc)

List of 7
 $ sdev    : Named num [1:12] 2.79 1.84 1.41 1.35 1.2 ...
  ..- attr(*, "names")= chr [1:12] "Comp.1" "Comp.2" "Comp.3" "Comp.4" ...
 $ loadings: loadings [1:12, 1:12] -0.315 -0.302 0.287 -0.383 -0.232 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:12] "baseline_b1_001" "baseline_b1_002" "baseline_b1_003" "baseline_b1_004" ...
  .. ..$ : chr [1:12] "Comp.1" "Comp.2" "Comp.3" "Comp.4" ...
 $ center  : Named num [1:12] 3.78 3.61 2.51 3.01 4.17 ...
  ..- attr(*, "names")= chr [1:12] "baseline_b1_001" "baseline_b1_002" "baseline_b1_003" "baseline_b1_004" ...
 $ scale   : Named num [1:12] 1 1 1 1 1 1 1 1 1 1 ...
  ..- attr(*, "names")= chr [1:12] "baseline_b1_001" "baseline_b1_002" "baseline_b1_003" "baseline_b1_004" ...
 $ n.obs   : int 390
 $ scores  : num [1:390, 1:12] 0.816 0.68 1.588 1.016 2.861 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:390] "1" "2" "3" "4" ...
  .. ..$ : chr [1:12] "Comp.1" "Comp.2" "Comp.3" "Comp.4" ...
 $ call    : language princ

## Percent of variance

In [10]:
100*pc$sdev^2/sum(pc$sdev^2)     #percent of variance

Comp.1    Comp.2    Comp.3    Comp.4    Comp.5    Comp.6    Comp.7    Comp.8 
37.951481 16.563109  9.678917  8.845460  7.053355  4.471337  3.766312  3.122614 
   Comp.9   Comp.10   Comp.11   Comp.12 
 2.722409  2.682021  1.875942  1.267042

## Loadings

In [11]:
pc$loadings


Loadings:
                Comp.1 Comp.2 Comp.3 Comp.4 Comp.5 Comp.6 Comp.7 Comp.8 Comp.9
baseline_b1_001 -0.315 -0.388  0.119  0.255                0.537  0.134  0.298
baseline_b1_002 -0.302 -0.366  0.228  0.238  0.247 -0.320 -0.136 -0.162 -0.236
baseline_b1_003  0.287  0.326 -0.140         0.521 -0.288  0.318 -0.353  0.392
baseline_b1_004 -0.383 -0.269 -0.450  0.117 -0.212        -0.163 -0.333  0.333
baseline_c1_001 -0.232               -0.324  0.334         0.313  0.172       
baseline_c1_002 -0.270         0.221 -0.334  0.393 -0.217 -0.287  0.155 -0.125
baseline_c1_003  0.211        -0.281  0.532  0.178 -0.220               -0.572
baseline_c1_004 -0.318        -0.541 -0.362  0.145  0.157        -0.296 -0.324
baseline_d1_001 -0.320  0.352  0.112  0.298         0.586  0.264        -0.187
baseline_d1_002 -0.282  0.384  0.294  0.302  0.121        -0.390 -0.328  0.173
baseline_d1_003  0.185 -0.289 -0.266  0.147  0.521  0.463 -0.383  0.296  0.237
baseline_d1_004 -0.303  0.412 -0.343  0.1

## Add scores to dataframe and set up Cox regression flag

In [12]:
pc1 = pc$scores[,1]
pc2 = pc$scores[,2]
pc3 = pc$scores[,3]
pc4 = pc$scores[,4]
perceived_severity_cox = cbind(perceived_severity,pc1,pc2,pc3,pc4)
str(perceived_severity_cox)

'data.frame':	390 obs. of  19 variables:
 $ isg_no         : num  10587 10636 10657 10664 10671 ...
 $ event          : num  1 1 1 0 1 1 0 1 1 1 ...
 $ hours_to_event : num  25.62 25.94 2.98 2088 114.74 ...
 $ baseline_b1_001: num  5 4 4 2 3 5 5 4 3 3 ...
 $ baseline_b1_002: num  1 4 3 2 3 5 5 2 3 3 ...
 $ baseline_b1_003: num  3 3 4 4 3 1 1 2 4 1 ...
 $ baseline_b1_004: num  4 3 3 2 3 5 5 3 2 5 ...
 $ baseline_c1_001: num  4 4 4 4 3 5 5 4 3 3 ...
 $ baseline_c1_002: num  3 3 3 4 3 5 5 4 3 4 ...
 $ baseline_c1_003: num  3 3 3 2 2 1 1 1 3 1 ...
 $ baseline_c1_004: num  2 3 3 4 3 5 4 1 3 5 ...
 $ baseline_d1_001: num  4 4 3 4 1 4 4 5 2 5 ...
 $ baseline_d1_002: num  2 3 3 4 2 4 3 4 2 5 ...
 $ baseline_d1_003: num  1 3 3 2 3 4 3 2 3 1 ...
 $ baseline_d1_004: num  4 3 3 4 2 5 3 4 2 5 ...
 $ pc1            : num  0.816 0.68 1.588 1.016 2.861 ...
 $ pc2            : num  0.6195 -0.3429 -0.0026 2.9911 -1.3518 ...
 $ pc3            : num  -0.838 -0.261 -0.741 -0.468 -0.681 ...
 $ pc4          

## Cox regression on first four principal component scores

In [13]:
cr = coxph(Surv(hours_to_event,event) ~ pc1+pc2+pc3+pc4,data=perceived_severity_cox,method="efron")
cr
summary(cr)

Call:
coxph(formula = Surv(hours_to_event, event) ~ pc1 + pc2 + pc3 + 
    pc4, data = perceived_severity_cox, method = "efron")

        coef exp(coef) se(coef)     z      p
pc1  0.03053   1.03100  0.01990  1.53 0.1249
pc2 -0.09301   0.91119  0.03041 -3.06 0.0022
pc3 -0.00324   0.99676  0.03926 -0.08 0.9342
pc4  0.07056   1.07311  0.04152  1.70 0.0892

Likelihood ratio test=13.9  on 4 df, p=0.00749
n= 390, number of events= 310 

Call:
coxph(formula = Surv(hours_to_event, event) ~ pc1 + pc2 + pc3 + 
    pc4, data = perceived_severity_cox, method = "efron")

  n= 390, number of events= 310 

         coef exp(coef)  se(coef)      z Pr(>|z|)   
pc1  0.030532  1.031002  0.019898  1.534  0.12493   
pc2 -0.093007  0.911187  0.030408 -3.059  0.00222 **
pc3 -0.003243  0.996762  0.039265 -0.083  0.93417   
pc4  0.070561  1.073110  0.041521  1.699  0.08924 . 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

    exp(coef) exp(-coef) lower .95 upper .95
pc1    1.0310     0.9699    0.9916    1.0720
pc2    0.9112     1.0975    0.8585    0.9671
pc3    0.9968     1.0032    0.9229    1.0765
pc4    1.0731     0.9319    0.9892    1.1641

Concordance= 0.568  (se = 0.018 )
Rsquare= 0.035   (max possible= 1 )
Likelihood ratio test= 13.94  on 4 df,   p=0.007495
Wald test            = 13.67  on 4 df,   p=0.008444
Score (logrank) test = 13.72  on 4 df,   p=0.008234


## Cox regression with principal component scores - quit intentions

## Compute principal components for baseline predictors

In [14]:
keeplist = c('isg_no','event','hours_to_event',
            'baseline_f1_002','baseline_f1_004','baseline_f1_006','baseline_f1_007','baseline_f1_008',
            'baseline_f2_002','baseline_f2_004','baseline_f2_006','baseline_f2_007','baseline_f2_008')

quit_intentions = na.omit(coxreg[,keeplist])
str(quit_intentions)

'data.frame':	389 obs. of  13 variables:
 $ isg_no         : num  10587 10636 10657 10664 10671 ...
 $ event          : num  1 1 1 0 1 1 0 1 1 1 ...
 $ hours_to_event : num  25.62 25.94 2.98 2088 114.74 ...
 $ baseline_f1_002: num  2 4 4 2 3 4 2 4 3 5 ...
 $ baseline_f1_004: num  3 4 4 2 3 5 2 4 3 5 ...
 $ baseline_f1_006: num  1 3 4 2 2 4 2 4 3 5 ...
 $ baseline_f1_007: num  1 4 4 2 2 5 2 4 3 5 ...
 $ baseline_f1_008: num  1 3 2 2 3 3 4 1 3 1 ...
 $ baseline_f2_002: num  3 4 4 4 4 5 5 5 3 5 ...
 $ baseline_f2_004: num  3 4 4 4 4 5 5 5 3 5 ...
 $ baseline_f2_006: num  4 3 4 4 3 5 5 4 3 5 ...
 $ baseline_f2_007: num  1 3 4 4 3 5 5 3 3 5 ...
 $ baseline_f2_008: num  5 3 3 1 2 1 1 1 3 1 ...
 - attr(*, "na.action")=Class 'omit'  Named int [1:45] 12 84 113 116 229 395 396 397 398 399 ...
  .. ..- attr(*, "names")= chr [1:45] "12" "84" "113" "116" ...


In [15]:
quit_intentions2 = quit_intentions[,4:13]
str(quit_intentions2)

'data.frame':	389 obs. of  10 variables:
 $ baseline_f1_002: num  2 4 4 2 3 4 2 4 3 5 ...
 $ baseline_f1_004: num  3 4 4 2 3 5 2 4 3 5 ...
 $ baseline_f1_006: num  1 3 4 2 2 4 2 4 3 5 ...
 $ baseline_f1_007: num  1 4 4 2 2 5 2 4 3 5 ...
 $ baseline_f1_008: num  1 3 2 2 3 3 4 1 3 1 ...
 $ baseline_f2_002: num  3 4 4 4 4 5 5 5 3 5 ...
 $ baseline_f2_004: num  3 4 4 4 4 5 5 5 3 5 ...
 $ baseline_f2_006: num  4 3 4 4 3 5 5 4 3 5 ...
 $ baseline_f2_007: num  1 3 4 4 3 5 5 3 3 5 ...
 $ baseline_f2_008: num  5 3 3 1 2 1 1 1 3 1 ...


In [16]:
summary(quit_intentions2)

 baseline_f1_002 baseline_f1_004 baseline_f1_006 baseline_f1_007
 Min.   :1.000   Min.   :1.000   Min.   :1.000   Min.   :1.00   
 1st Qu.:1.000   1st Qu.:2.000   1st Qu.:1.000   1st Qu.:2.00   
 Median :3.000   Median :3.000   Median :3.000   Median :3.00   
 Mean   :2.753   Mean   :3.149   Mean   :2.674   Mean   :3.08   
 3rd Qu.:4.000   3rd Qu.:4.000   3rd Qu.:4.000   3rd Qu.:4.00   
 Max.   :5.000   Max.   :5.000   Max.   :5.000   Max.   :5.00   
 baseline_f1_008 baseline_f2_002 baseline_f2_004 baseline_f2_006
 Min.   :1.000   Min.   :1.000   Min.   :1.000   Min.   :1.000  
 1st Qu.:2.000   1st Qu.:3.000   1st Qu.:3.000   1st Qu.:2.000  
 Median :3.000   Median :4.000   Median :4.000   Median :4.000  
 Mean   :2.792   Mean   :3.882   Mean   :3.859   Mean   :3.365  
 3rd Qu.:4.000   3rd Qu.:5.000   3rd Qu.:5.000   3rd Qu.:5.000  
 Max.   :5.000   Max.   :5.000   Max.   :5.000   Max.   :5.000  
 baseline_f2_007 baseline_f2_008
 Min.   :1.000   Min.   :1.000  
 1st Qu.:2.000   1st Qu.

In [17]:
quit_intentions2

,baseline_f1_002,baseline_f1_004,baseline_f1_006,baseline_f1_007,baseline_f1_008,baseline_f2_002,baseline_f2_004,baseline_f2_006,baseline_f2_007,baseline_f2_008
1,2,3,1,1,1,3,3,4,1,5
2,4,4,3,4,3,4,4,3,3,3
3,4,4,4,4,2,4,4,4,4,3
4,2,2,2,2,2,4,4,4,4,1
5,3,3,2,2,3,4,4,3,3,2
6,4,5,4,5,3,5,5,5,5,1
7,2,2,2,2,4,5,5,5,5,1
8,4,4,4,4,1,5,5,4,3,1
9,3,3,3,3,3,3,3,3,3,3
10,5,5,5,5,1,5,5,5,5,1


In [18]:
pc = princomp(quit_intentions2,scores=TRUE)

In [19]:
str(pc)

List of 7
 $ sdev    : Named num [1:10] 3.01 1.72 1.3 1.07 1.04 ...
  ..- attr(*, "names")= chr [1:10] "Comp.1" "Comp.2" "Comp.3" "Comp.4" ...
 $ loadings: loadings [1:10, 1:10] -0.231 -0.359 -0.329 -0.35 0.302 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:10] "baseline_f1_002" "baseline_f1_004" "baseline_f1_006" "baseline_f1_007" ...
  .. ..$ : chr [1:10] "Comp.1" "Comp.2" "Comp.3" "Comp.4" ...
 $ center  : Named num [1:10] 2.75 3.15 2.67 3.08 2.79 ...
  ..- attr(*, "names")= chr [1:10] "baseline_f1_002" "baseline_f1_004" "baseline_f1_006" "baseline_f1_007" ...
 $ scale   : Named num [1:10] 1 1 1 1 1 1 1 1 1 1
  ..- attr(*, "names")= chr [1:10] "baseline_f1_002" "baseline_f1_004" "baseline_f1_006" "baseline_f1_007" ...
 $ n.obs   : int 389
 $ scores  : num [1:389, 1:10] 2.7956 -0.6126 -1.9002 0.0473 0.7107 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:389] "1" "2" "3" "4" ...
  .. ..$ : chr [1:10] "Comp.1" "Comp.2" "Comp.3" "Comp.4" ...
 $ call    : language pr

## Percent of variance

In [20]:
100*pc$sdev^2/sum(pc$sdev^2)     #percent of variance

Comp.1    Comp.2    Comp.3    Comp.4    Comp.5    Comp.6    Comp.7    Comp.8 
47.837585 15.744278  8.895223  6.052256  5.752479  5.243069  4.182043  2.384324 
   Comp.9   Comp.10 
 2.099324  1.809420

## Loadings

In [21]:
pc$loadings


Loadings:
                Comp.1 Comp.2 Comp.3 Comp.4 Comp.5 Comp.6 Comp.7 Comp.8 Comp.9
baseline_f1_002 -0.231 -0.431 -0.190  0.304  0.666  0.323        -0.228 -0.105
baseline_f1_004 -0.359 -0.276  0.160  0.391 -0.320 -0.271 -0.514 -0.288  0.257
baseline_f1_006 -0.329 -0.305 -0.485 -0.368 -0.143        -0.336  0.522 -0.141
baseline_f1_007 -0.350 -0.339  0.231  0.117 -0.368  0.130  0.612  0.207  0.169
baseline_f1_008  0.302  0.176 -0.544  0.536 -0.135                0.187  0.304
baseline_f2_002 -0.305  0.230         0.243  0.327 -0.242  0.268  0.401  0.253
baseline_f2_004 -0.317  0.268         0.245        -0.447               -0.688
baseline_f2_006 -0.346  0.197 -0.470 -0.363        -0.203  0.254 -0.551  0.266
baseline_f2_007 -0.310  0.455         0.189 -0.278  0.705               -0.169
baseline_f2_008  0.295 -0.359 -0.351  0.177 -0.308         0.321 -0.201 -0.381
                Comp.10
baseline_f1_002 -0.142 
baseline_f1_004  0.168 
baseline_f1_006        
baseline_f1_007 -0.313 


## Add scores to dataframe and set up Cox regression flag

In [22]:
pc1 = pc$scores[,1]
pc2 = pc$scores[,2]
pc3 = pc$scores[,3]
pc4 = pc$scores[,4]
quit_intentions_cox = cbind(quit_intentions,pc1,pc2,pc3,pc4)
str(quit_intentions_cox)

'data.frame':	389 obs. of  17 variables:
 $ isg_no         : num  10587 10636 10657 10664 10671 ...
 $ event          : num  1 1 1 0 1 1 0 1 1 1 ...
 $ hours_to_event : num  25.62 25.94 2.98 2088 114.74 ...
 $ baseline_f1_002: num  2 4 4 2 3 4 2 4 3 5 ...
 $ baseline_f1_004: num  3 4 4 2 3 5 2 4 3 5 ...
 $ baseline_f1_006: num  1 3 4 2 2 4 2 4 3 5 ...
 $ baseline_f1_007: num  1 4 4 2 2 5 2 4 3 5 ...
 $ baseline_f1_008: num  1 3 2 2 3 3 4 1 3 1 ...
 $ baseline_f2_002: num  3 4 4 4 4 5 5 5 3 5 ...
 $ baseline_f2_004: num  3 4 4 4 4 5 5 5 3 5 ...
 $ baseline_f2_006: num  4 3 4 4 3 5 5 4 3 5 ...
 $ baseline_f2_007: num  1 3 4 4 3 5 5 3 3 5 ...
 $ baseline_f2_008: num  5 3 3 1 2 1 1 1 3 1 ...
 $ pc1            : num  2.7956 -0.6126 -1.9002 0.0473 0.7107 ...
 $ pc2            : num  -1.061 -1.542 -1.371 2.048 0.506 ...
 $ pc3            : num  0.386 -0.23 -0.712 0.559 0.175 ...
 $ pc4            : num  -1.47597 1.08218 0.00309 -1.23922 0.34417 ...


## Cox regression on first four principal component scores

In [23]:
cr = coxph(Surv(hours_to_event,event) ~ pc1+pc2+pc3+pc4,data=quit_intentions_cox,method="efron")
cr
summary(cr)

Call:
coxph(formula = Surv(hours_to_event, event) ~ pc1 + pc2 + pc3 + 
    pc4, data = quit_intentions_cox, method = "efron")

       coef exp(coef) se(coef)     z       p
pc1  0.0860    1.0899   0.0182  4.72 2.4e-06
pc2 -0.2413    0.7856   0.0361 -6.69 2.2e-11
pc3  0.0305    1.0310   0.0462  0.66    0.51
pc4  0.0479    1.0490   0.0516  0.93    0.35

Likelihood ratio test=66.6  on 4 df, p=1.19e-13
n= 389, number of events= 309 

Call:
coxph(formula = Surv(hours_to_event, event) ~ pc1 + pc2 + pc3 + 
    pc4, data = quit_intentions_cox, method = "efron")

  n= 389, number of events= 309 

        coef exp(coef) se(coef)      z Pr(>|z|)    
pc1  0.08605   1.08986  0.01825  4.716 2.41e-06 ***
pc2 -0.24131   0.78560  0.03607 -6.690 2.23e-11 ***
pc3  0.03052   1.03099  0.04618  0.661    0.509    
pc4  0.04787   1.04903  0.05161  0.928    0.354    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

    exp(coef) exp(-coef) lower .95 upper .95
pc1    1.0899     0.9176    1.0516    1.1295
pc2    0.7856     1.2729    0.7320    0.8431
pc3    1.0310     0.9699    0.9418    1.1287
pc4    1.0490     0.9533    0.9481    1.1607

Concordance= 0.637  (se = 0.018 )
Rsquare= 0.157   (max possible= 1 )
Likelihood ratio test= 66.59  on 4 df,   p=1.188e-13
Wald test            = 65.1  on 4 df,   p=2.455e-13
Score (logrank) test = 66.99  on 4 df,   p=9.781e-14


## Cox regression with principal components and Leikert scale scores - event related fear

## Compute principal components for baseline predictors

In [24]:
keeplist = c('isg_no','event','hours_to_event',
            'baseline_b2_002','baseline_c2_002','baseline_d2_002')

event_related_fear = na.omit(coxreg[,keeplist])
str(event_related_fear)

'data.frame':	391 obs. of  6 variables:
 $ isg_no         : num  10587 10636 10657 10664 10671 ...
 $ event          : num  1 1 1 0 1 1 0 1 1 1 ...
 $ hours_to_event : num  25.62 25.94 2.98 2088 114.74 ...
 $ baseline_b2_002: num  4 4 4 2 4 4 4 2 3 5 ...
 $ baseline_c2_002: num  4 3 4 2 4 4 4 1 4 4 ...
 $ baseline_d2_002: num  1 2 5 1 3 3 2 1 1 5 ...
 - attr(*, "na.action")=Class 'omit'  Named int [1:43] 12 84 113 395 396 397 398 399 400 401 ...
  .. ..- attr(*, "names")= chr [1:43] "12" "84" "113" "395" ...


In [25]:
event_related_fear2 = event_related_fear[,4:6]
str(event_related_fear2)

'data.frame':	391 obs. of  3 variables:
 $ baseline_b2_002: num  4 4 4 2 4 4 4 2 3 5 ...
 $ baseline_c2_002: num  4 3 4 2 4 4 4 1 4 4 ...
 $ baseline_d2_002: num  1 2 5 1 3 3 2 1 1 5 ...


In [26]:
summary(event_related_fear2)

 baseline_b2_002 baseline_c2_002 baseline_d2_002
 Min.   :1.000   Min.   :1.000   Min.   :1.000  
 1st Qu.:1.000   1st Qu.:1.000   1st Qu.:1.000  
 Median :3.000   Median :3.000   Median :1.000  
 Mean   :2.887   Mean   :2.962   Mean   :2.087  
 3rd Qu.:4.000   3rd Qu.:5.000   3rd Qu.:3.000  
 Max.   :5.000   Max.   :5.000   Max.   :5.000  

In [27]:
event_related_fear2

,baseline_b2_002,baseline_c2_002,baseline_d2_002
1,4,4,1
2,4,3,2
3,4,4,5
4,2,2,1
5,4,4,3
6,4,4,3
7,4,4,2
8,2,1,1
9,3,4,1
10,5,4,5


In [28]:
pc = princomp(event_related_fear2,scores=TRUE)

In [29]:
str(pc)

List of 7
 $ sdev    : Named num [1:3] 2.15 1.17 1.01
  ..- attr(*, "names")= chr [1:3] "Comp.1" "Comp.2" "Comp.3"
 $ loadings: loadings [1:3, 1:3] -0.6048 -0.6389 -0.4754 0.5974 0.0307 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:3] "baseline_b2_002" "baseline_c2_002" "baseline_d2_002"
  .. ..$ : chr [1:3] "Comp.1" "Comp.2" "Comp.3"
 $ center  : Named num [1:3] 2.89 2.96 2.09
  ..- attr(*, "names")= chr [1:3] "baseline_b2_002" "baseline_c2_002" "baseline_d2_002"
 $ scale   : Named num [1:3] 1 1 1
  ..- attr(*, "names")= chr [1:3] "baseline_b2_002" "baseline_c2_002" "baseline_d2_002"
 $ n.obs   : int 391
 $ scores  : num [1:391, 1:3] -0.82 -0.656 -2.721 1.668 -1.77 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:391] "1" "2" "3" "4" ...
  .. ..$ : chr [1:3] "Comp.1" "Comp.2" "Comp.3"
 $ call    : language princomp(x = event_related_fear2, scores = TRUE)
 - attr(*, "class")= chr "princomp"


## Percent of variance

In [30]:
100*pc$sdev^2/sum(pc$sdev^2)     #percent of variance

Comp.1   Comp.2   Comp.3 
65.77469 19.67912 14.54619

## Loadings

In [31]:
pc$loadings


Loadings:
                Comp.1 Comp.2 Comp.3
baseline_b2_002 -0.605  0.597  0.527
baseline_c2_002 -0.639        -0.769
baseline_d2_002 -0.475 -0.801  0.363

               Comp.1 Comp.2 Comp.3
SS loadings     1.000  1.000  1.000
Proportion Var  0.333  0.333  0.333
Cumulative Var  0.333  0.667  1.000

## Add scores to dataframe and set up Cox regression flag

In [32]:
pc1 = pc$scores[,1]
pc2 = pc$scores[,2]
pc3 = pc$scores[,3]
event_related_fear_cox = cbind(event_related_fear,pc1,pc2,pc3)
str(event_related_fear_cox)

'data.frame':	391 obs. of  9 variables:
 $ isg_no         : num  10587 10636 10657 10664 10671 ...
 $ event          : num  1 1 1 0 1 1 0 1 1 1 ...
 $ hours_to_event : num  25.62 25.94 2.98 2088 114.74 ...
 $ baseline_b2_002: num  4 4 4 2 4 4 4 2 3 5 ...
 $ baseline_c2_002: num  4 3 4 2 4 4 4 1 4 4 ...
 $ baseline_d2_002: num  1 2 5 1 3 3 2 1 1 5 ...
 $ pc1            : num  -0.82 -0.656 -2.721 1.668 -1.77 ...
 $ pc2            : num  1.5675 0.7355 -1.6379 0.3114 -0.0352 ...
 $ pc3            : num  -0.607 0.525 0.845 -0.123 0.119 ...


## Cox regression on first three principal component scores

In [33]:
cr = coxph(Surv(hours_to_event,event) ~ pc1+pc2+pc3,data=event_related_fear_cox,method="efron")
cr
summary(cr)

Call:
coxph(formula = Surv(hours_to_event, event) ~ pc1 + pc2 + pc3, 
    data = event_related_fear_cox, method = "efron")

        coef exp(coef) se(coef)     z     p
pc1 -0.00567   0.99435  0.02622 -0.22 0.829
pc2  0.07801   1.08113  0.04803  1.62 0.104
pc3  0.10152   1.10685  0.05591  1.82 0.069

Likelihood ratio test=6.41  on 3 df, p=0.0935
n= 391, number of events= 311 

Call:
coxph(formula = Surv(hours_to_event, event) ~ pc1 + pc2 + pc3, 
    data = event_related_fear_cox, method = "efron")

  n= 391, number of events= 311 

         coef exp(coef)  se(coef)      z Pr(>|z|)  
pc1 -0.005668  0.994348  0.026224 -0.216   0.8289  
pc2  0.078008  1.081131  0.048030  1.624   0.1043  
pc3  0.101521  1.106853  0.055911  1.816   0.0694 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

    exp(coef) exp(-coef) lower .95 upper .95
pc1    0.9943     1.0057    0.9445     1.047
pc2    1.0811     0.9250    0.9840     1.188
pc3    1.1069     0.9035    0.9920     1.235

Concordance= 0.537  (se = 0.018 )
Rsquare= 0.016   (max possible= 1 )
Likelihood ratio test= 6.41  on 3 df,   p=0.09348
Wald test            = 6.43  on 3 df,   p=0.09251
Score (logrank) test = 6.4  on 3 df,   p=0.09353


### Cox regression on Leikert scale scores

In [34]:
cr = coxph(Surv(hours_to_event,event) ~ baseline_b2_002+baseline_c2_002+baseline_d2_002,data=event_related_fear_cox,method="efron")
cr
summary(cr)

Call:
coxph(formula = Surv(hours_to_event, event) ~ baseline_b2_002 + 
    baseline_c2_002 + baseline_d2_002, data = event_related_fear_cox, 
    method = "efron")

                   coef exp(coef) se(coef)     z     p
baseline_b2_002  0.1035    1.1090   0.0429  2.41 0.016
baseline_c2_002 -0.0720    0.9305   0.0461 -1.56 0.118
baseline_d2_002 -0.0230    0.9773   0.0465 -0.49 0.621

Likelihood ratio test=6.41  on 3 df, p=0.0935
n= 391, number of events= 311 

Call:
coxph(formula = Surv(hours_to_event, event) ~ baseline_b2_002 + 
    baseline_c2_002 + baseline_d2_002, data = event_related_fear_cox, 
    method = "efron")

  n= 391, number of events= 311 

                    coef exp(coef) se(coef)      z Pr(>|z|)  
baseline_b2_002  0.10349   1.10903  0.04291  2.412   0.0159 *
baseline_c2_002 -0.07202   0.93051  0.04606 -1.564   0.1179  
baseline_d2_002 -0.02296   0.97730  0.04645 -0.494   0.6212  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

                exp(coef) exp(-coef) lower .95 upper .95
baseline_b2_002    1.1090     0.9017    1.0196     1.206
baseline_c2_002    0.9305     1.0747    0.8502     1.018
baseline_d2_002    0.9773     1.0232    0.8923     1.070

Concordance= 0.537  (se = 0.018 )
Rsquare= 0.016   (max possible= 1 )
Likelihood ratio test= 6.41  on 3 df,   p=0.09348
Wald test            = 6.43  on 3 df,   p=0.09251
Score (logrank) test = 6.4  on 3 df,   p=0.09353


## Cox regression with principal components and Leikert scale scores - causal attribution

## Compute principal components for baseline predictors

In [35]:
keeplist = c('isg_no','event','hours_to_event',
            'baseline_e1_004','baseline_e2_001','baseline_e2_002','baseline_e2_003','baseline_e2_004')

causal_attribution = na.omit(coxreg[,keeplist])
str(causal_attribution)

'data.frame':	389 obs. of  8 variables:
 $ isg_no         : num  10587 10636 10657 10664 10671 ...
 $ event          : num  1 1 1 0 1 1 0 1 1 1 ...
 $ hours_to_event : num  25.62 25.94 2.98 2088 114.74 ...
 $ baseline_e1_004: num  5 3 4 4 3 5 5 4 3 3 ...
 $ baseline_e2_001: num  2 5 3 3 3 5 5 5 3 5 ...
 $ baseline_e2_002: num  2 5 3 2 3 5 5 5 3 5 ...
 $ baseline_e2_003: num  4 5 3 3 5 5 5 5 4 5 ...
 $ baseline_e2_004: num  3 4 3 3 3 5 5 4 4 4 ...
 - attr(*, "na.action")=Class 'omit'  Named int [1:45] 12 68 84 87 113 395 396 397 398 399 ...
  .. ..- attr(*, "names")= chr [1:45] "12" "68" "84" "87" ...


In [36]:
causal_attribution2 = causal_attribution[,4:8]
str(causal_attribution2)

'data.frame':	389 obs. of  5 variables:
 $ baseline_e1_004: num  5 3 4 4 3 5 5 4 3 3 ...
 $ baseline_e2_001: num  2 5 3 3 3 5 5 5 3 5 ...
 $ baseline_e2_002: num  2 5 3 2 3 5 5 5 3 5 ...
 $ baseline_e2_003: num  4 5 3 3 5 5 5 5 4 5 ...
 $ baseline_e2_004: num  3 4 3 3 3 5 5 4 4 4 ...


In [37]:
summary(causal_attribution2)

 baseline_e1_004 baseline_e2_001 baseline_e2_002 baseline_e2_003
 Min.   :1.000   Min.   :1.00    Min.   :1.000   Min.   :1.000  
 1st Qu.:4.000   1st Qu.:3.00    1st Qu.:3.000   1st Qu.:4.000  
 Median :4.000   Median :4.00    Median :4.000   Median :5.000  
 Mean   :3.913   Mean   :3.54    Mean   :3.841   Mean   :4.442  
 3rd Qu.:5.000   3rd Qu.:5.00    3rd Qu.:5.000   3rd Qu.:5.000  
 Max.   :5.000   Max.   :5.00    Max.   :5.000   Max.   :5.000  
 baseline_e2_004
 Min.   :1.000  
 1st Qu.:3.000  
 Median :4.000  
 Mean   :3.594  
 3rd Qu.:5.000  
 Max.   :5.000  

In [38]:
causal_attribution2

,baseline_e1_004,baseline_e2_001,baseline_e2_002,baseline_e2_003,baseline_e2_004
1,5,2,2,4,3
2,3,5,5,5,4
3,4,3,3,3,3
4,4,3,2,3,3
5,3,3,3,5,3
6,5,5,5,5,5
7,5,5,5,5,5
8,4,5,5,5,4
9,3,3,3,4,4
10,3,5,5,5,4


In [39]:
pc = princomp(causal_attribution2,scores=TRUE)

In [40]:
str(pc)

List of 7
 $ sdev    : Named num [1:5] 2.212 0.946 0.83 0.709 0.65
  ..- attr(*, "names")= chr [1:5] "Comp.1" "Comp.2" "Comp.3" "Comp.4" ...
 $ loadings: loadings [1:5, 1:5] -0.457 -0.56 -0.487 -0.167 -0.461 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:5] "baseline_e1_004" "baseline_e2_001" "baseline_e2_002" "baseline_e2_003" ...
  .. ..$ : chr [1:5] "Comp.1" "Comp.2" "Comp.3" "Comp.4" ...
 $ center  : Named num [1:5] 3.91 3.54 3.84 4.44 3.59
  ..- attr(*, "names")= chr [1:5] "baseline_e1_004" "baseline_e2_001" "baseline_e2_002" "baseline_e2_003" ...
 $ scale   : Named num [1:5] 1 1 1 1 1
  ..- attr(*, "names")= chr [1:5] "baseline_e1_004" "baseline_e2_001" "baseline_e2_002" "baseline_e2_003" ...
 $ n.obs   : int 389
 $ scores  : num [1:389, 1:5] 1.61 -1.25 1.19 1.67 1.31 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:389] "1" "2" "3" "4" ...
  .. ..$ : chr [1:5] "Comp.1" "Comp.2" "Comp.3" "Comp.4" ...
 $ call    : language princomp(x = causal_attribution2, scor

## Percent of variance

In [41]:
100*pc$sdev^2/sum(pc$sdev^2)     #percent of variance

Comp.1    Comp.2    Comp.3    Comp.4    Comp.5 
66.127147 12.078337  9.298318  6.789948  5.706251

## Loadings

In [42]:
pc$loadings


Loadings:
                Comp.1 Comp.2 Comp.3 Comp.4 Comp.5
baseline_e1_004 -0.457         0.858  0.217       
baseline_e2_001 -0.560  0.105 -0.476  0.644  0.183
baseline_e2_002 -0.487        -0.159 -0.310 -0.801
baseline_e2_003 -0.167 -0.977                0.133
baseline_e2_004 -0.461  0.163 -0.108 -0.665  0.554

               Comp.1 Comp.2 Comp.3 Comp.4 Comp.5
SS loadings       1.0    1.0    1.0    1.0    1.0
Proportion Var    0.2    0.2    0.2    0.2    0.2
Cumulative Var    0.2    0.4    0.6    0.8    1.0

## Add scores to dataframe and set up Cox regression flag

In [43]:
pc1 = pc$scores[,1]
pc2 = pc$scores[,2]
pc3 = pc$scores[,3]
causal_attribution_cox = cbind(causal_attribution,pc1,pc2,pc3)
str(causal_attribution_cox)

'data.frame':	389 obs. of  11 variables:
 $ isg_no         : num  10587 10636 10657 10664 10671 ...
 $ event          : num  1 1 1 0 1 1 0 1 1 1 ...
 $ hours_to_event : num  25.62 25.94 2.98 2088 114.74 ...
 $ baseline_e1_004: num  5 3 4 4 3 5 5 4 3 3 ...
 $ baseline_e2_001: num  2 5 3 3 3 5 5 5 3 5 ...
 $ baseline_e2_002: num  2 5 3 2 3 5 5 5 3 5 ...
 $ baseline_e2_003: num  4 5 3 3 5 5 5 5 4 5 ...
 $ baseline_e2_004: num  3 4 3 3 3 5 5 4 4 4 ...
 $ pc1            : num  1.61 -1.25 1.19 1.67 1.31 ...
 $ pc2            : num  0.311 -0.432 1.282 1.304 -0.76 ...
 $ pc3            : num  2.018 -1.7 0.513 0.672 -0.321 ...


## Cox regression on first three principal component scores

In [44]:
cr = coxph(Surv(hours_to_event,event) ~ pc1+pc2+pc3,data=causal_attribution_cox,method="efron")
cr
summary(cr)

Call:
coxph(formula = Surv(hours_to_event, event) ~ pc1 + pc2 + pc3, 
    data = causal_attribution_cox, method = "efron")

      coef exp(coef) se(coef)    z      p
pc1 0.0745    1.0774   0.0247 3.02 0.0026
pc2 0.0344    1.0350   0.0575 0.60 0.5492
pc3 0.0344    1.0350   0.0693 0.50 0.6201

Likelihood ratio test=9.16  on 3 df, p=0.0273
n= 389, number of events= 309 

Call:
coxph(formula = Surv(hours_to_event, event) ~ pc1 + pc2 + pc3, 
    data = causal_attribution_cox, method = "efron")

  n= 389, number of events= 309 

       coef exp(coef) se(coef)     z Pr(>|z|)   
pc1 0.07452   1.07736  0.02470 3.016  0.00256 **
pc2 0.03445   1.03505  0.05752 0.599  0.54924   
pc3 0.03437   1.03497  0.06933 0.496  0.62011   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

    exp(coef) exp(-coef) lower .95 upper .95
pc1     1.077     0.9282    1.0264     1.131
pc2     1.035     0.9661    0.9247     1.159
pc3     1.035     0.9662    0.9035     1.186

Concordance= 0.557  (se = 0.018 )
Rsquare= 0.023   (max possible= 1 )
Likelihood ratio test= 9.16  on 3 df,   p=0.02729
Wald test            = 9.54  on 3 df,   p=0.0229
Score (logrank) test = 9.59  on 3 df,   p=0.02238


### Cox regression on Leikert scale scores

In [45]:
cr = coxph(Surv(hours_to_event,event) ~ baseline_e1_004+baseline_e2_001+baseline_e2_002+baseline_e2_003+baseline_e2_004,data=causal_attribution_cox,method="efron")
cr
summary(cr)

Call:
coxph(formula = Surv(hours_to_event, event) ~ baseline_e1_004 + 
    baseline_e2_001 + baseline_e2_002 + baseline_e2_003 + baseline_e2_004, 
    data = causal_attribution_cox, method = "efron")

                   coef exp(coef) se(coef)     z     p
baseline_e1_004 -0.0214    0.9788   0.0645 -0.33 0.740
baseline_e2_001 -0.1027    0.9024   0.0611 -1.68 0.093
baseline_e2_002  0.0491    1.0504   0.0777  0.63 0.527
baseline_e2_003 -0.0578    0.9438   0.0576 -1.00 0.315
baseline_e2_004 -0.0496    0.9516   0.0735 -0.68 0.500

Likelihood ratio test=10.7  on 5 df, p=0.057
n= 389, number of events= 309 

Call:
coxph(formula = Surv(hours_to_event, event) ~ baseline_e1_004 + 
    baseline_e2_001 + baseline_e2_002 + baseline_e2_003 + baseline_e2_004, 
    data = causal_attribution_cox, method = "efron")

  n= 389, number of events= 309 

                    coef exp(coef) se(coef)      z Pr(>|z|)  
baseline_e1_004 -0.02138   0.97884  0.06452 -0.331    0.740  
baseline_e2_001 -0.10269   0.90240  0.06114 -1.680    0.093 .
baseline_e2_002  0.04914   1.05036  0.07770  0.632    0.527  
baseline_e2_003 -0.05780   0.94384  0.05757 -1.004    0.315  
baseline_e2_004 -0.04962   0.95159  0.07351 -0.675    0.500  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

                exp(coef) exp(-coef) lower .95 upper .95
baseline_e1_004    0.9788     1.0216    0.8626     1.111
baseline_e2_001    0.9024     1.1082    0.8005     1.017
baseline_e2_002    1.0504     0.9521    0.9020     1.223
baseline_e2_003    0.9438     1.0595    0.8431     1.057
baseline_e2_004    0.9516     1.0509    

## Cox regression - actual severity

In [46]:
cr = coxph(Surv(hours_to_event,event) ~ actual_severity,data=coxreg,method="efron")
cr
summary(cr)

Call:
coxph(formula = Surv(hours_to_event, event) ~ actual_severity, 
    data = coxreg, method = "efron")

                  coef exp(coef) se(coef)     z       p
actual_severity -0.758     0.469    0.107 -7.11 1.2e-12

Likelihood ratio test=51.4  on 1 df, p=7.54e-13
n= 394, number of events= 314 
   (40 observations deleted due to missingness)

Call:
coxph(formula = Surv(hours_to_event, event) ~ actual_severity, 
    data = coxreg, method = "efron")

  n= 394, number of events= 314 
   (40 observations deleted due to missingness)

                   coef exp(coef) se(coef)      z Pr(>|z|)    
actual_severity -0.7582    0.4685   0.1067 -7.105  1.2e-12 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

                exp(coef) exp(-coef) lower .95 upper .95
actual_severity    0.4685      2.134    0.3801    0.5775

Concordance= 0.599  (se = 0.015 )
Rsquare= 0.122   (max possible= 1 )
Likelihood ratio test= 51.4  on 1 df,   p=7.541e-13
Wald test            = 50.48  on 1 df,   p=1.202e-12
Score (logrank) test = 50.42  on 1 df,   p=1.242e-12


## Cox regression - TIMI

In [47]:
cr = coxph(Surv(hours_to_event,event) ~ TIMI,data=coxreg,method="efron")
cr
summary(cr)

Call:
coxph(formula = Surv(hours_to_event, event) ~ TIMI, data = coxreg, 
    method = "efron")

        coef exp(coef) se(coef)    z      p
TIMI -0.1180    0.8887   0.0454 -2.6 0.0094

Likelihood ratio test=6.85  on 1 df, p=0.00885
n= 394, number of events= 314 
   (40 observations deleted due to missingness)

Call:
coxph(formula = Surv(hours_to_event, event) ~ TIMI, data = coxreg, 
    method = "efron")

  n= 394, number of events= 314 
   (40 observations deleted due to missingness)

        coef exp(coef) se(coef)      z Pr(>|z|)   
TIMI -0.1180    0.8887   0.0454 -2.599  0.00936 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

     exp(coef) exp(-coef) lower .95 upper .95
TIMI    0.8887      1.125    0.8131    0.9714

Concordance= 0.547  (se = 0.017 )
Rsquare= 0.017   (max possible= 1 )
Likelihood ratio test= 6.85  on 1 df,   p=0.008852
Wald test            = 6.75  on 1 df,   p=0.009359
Score (logrank) test = 6.77  on 1 df,   p=0.00928
